# Create Categories for ArcGIS Online
The purpose of this notebook is to illustrate how to use the SDG Metadata API to create a series of categories in ArcGIS online for organizing and filtering data.

# Installation & Requirements
Data Package python lib
+ pip install ipdb
+ pip install requests
+ pip install urllib3


# ArcGIS API for Python
Install & Setup Guide here: https://developers.arcgis.com/python/guide/install-and-set-up 

### Create a connection to your ArcGIS Online Organization
This will rely on using the ArcGIS API for python to connect to your ArcGIS Online Organization to publish and manage data. For more information about this python library visit the developer resources at https://developers.arcgis.com/python/

In [66]:
import copy
# used to prompt for user input
# when using this script internally, you may remove this and simply hard code in your username and password
import getpass
import json
import os
import re
import sys
import time
import traceback
import urllib
import urllib.request as request
import urllib.request as urlopen
from datetime import datetime
import requests
# this helps us do some debugging within the Python Notebook
# another optional component
from IPython.display import display
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from arcgis.gis.server import Service

online_username = input('Username: ')
online_password = getpass.getpass('Password: ')
online_connection = "https://www.arcgis.com"
gis_online_connection = GIS(online_connection, online_username, online_password)
gis_online_connection

Username: tbutcher_undesa
Password: ········


### Get the JSON Data from the UN SDG API

The SDG API is designed to  retrieve information and metadata on the [Sustainable Development Goals](http://www.un.org/sustainabledevelopment/sustainable-development-goals/).

The Inter-agency Expert Group on SDG Indicators released a series of PDFs that includes metadata for each Indicator. Those PDFs can be downloaded [here](http://unstats.un.org/sdgs/iaeg-sdgs/metadata-compilation/).

In [67]:
url = "https://unstats.un.org/SDGAPI/v1/sdg/Goal/List?includechildren=true"
req = request.Request(url)
response = urlopen.urlopen(req)
response_bytes = response.read()
json_data = json.loads(response_bytes.decode("UTF-8"))

### Build the Category Schema File
Loop through the SDG Data to build a JSON object to apply as categories

In [68]:
catergory_schema = {"categorySchema":[{"title":"Categories","categories":[]}]}
catergory_schema["categorySchema"][0]["categories"]

for goal in json_data:
    goal_schema = {"title":"Goal " + goal["code"],"categories":[]}
    for target in goal["targets"]:
        target_schema = {"title":"Target " + target["code"],"categories":[]}
        for indicator in target["indicators"]:
            indicator_schema = {"title":"Indicator " + indicator["code"]}
            # We are currently only building out to the Target Level of the data
            #with a limitation on the nodes in ArcGIS Online.  Future versions will
            #build out to the indicator and series level
            #target_schema["categories"].append(indicator_schema)
        goal_schema["categories"].append(target_schema)
    catergory_schema["categorySchema"][0]["categories"].append(goal_schema)
catergory_schema       

{'categorySchema': [{'title': 'Categories',
   'categories': [{'title': 'Goal 1',
     'categories': [{'title': 'Target 1.1', 'categories': []},
      {'title': 'Target 1.2', 'categories': []},
      {'title': 'Target 1.3', 'categories': []},
      {'title': 'Target 1.4', 'categories': []},
      {'title': 'Target 1.5', 'categories': []},
      {'title': 'Target 1.a', 'categories': []},
      {'title': 'Target 1.b', 'categories': []}]},
    {'title': 'Goal 2',
     'categories': [{'title': 'Target 2.1', 'categories': []},
      {'title': 'Target 2.2', 'categories': []},
      {'title': 'Target 2.3', 'categories': []},
      {'title': 'Target 2.4', 'categories': []},
      {'title': 'Target 2.5', 'categories': []},
      {'title': 'Target 2.a', 'categories': []},
      {'title': 'Target 2.b', 'categories': []},
      {'title': 'Target 2.c', 'categories': []}]},
    {'title': 'Goal 3',
     'categories': [{'title': 'Target 3.1', 'categories': []},
      {'title': 'Target 3.2', 'categorie

### Apply the category schema
Use the category schema object to build the ArcGIS Online Categories.  This will set the categories for you organization, remember that other members of your organization may have already assigned these categories to their items, and editing the schema will update those items accordingly.

In [69]:
category_url = gis_online_connection._url + "/sharing/rest/portals/" + gis_online_connection.properties.id + "/assignCategorySchema"
category_params = {"f": "json", 
                   "token": gis_online_connection._con.token,
                   "categorySchema": json.dumps(catergory_schema)
                  }
print(category_params)
r = requests.post(category_url, data=category_params)
category_json_data = json.loads(r.content.decode("UTF-8"))
category_json_data

{'f': 'json', 'token': 'XLI3AwuAY5ZIJN01n_v8aCsUAL3raewlQyQGZs4Auc1rM70ZKyviCNcQAo6BO5SFd_K5HDpIRdlNqd8L3m2nEvcGqLvsEobD3u-LKlvqo18Dhu09nlIlARGu7gY9BbHQ', 'categorySchema': '{"categorySchema": [{"title": "Categories", "categories": [{"title": "Goal 1", "categories": [{"title": "Target 1.1", "categories": []}, {"title": "Target 1.2", "categories": []}, {"title": "Target 1.3", "categories": []}, {"title": "Target 1.4", "categories": []}, {"title": "Target 1.5", "categories": []}, {"title": "Target 1.a", "categories": []}, {"title": "Target 1.b", "categories": []}]}, {"title": "Goal 2", "categories": [{"title": "Target 2.1", "categories": []}, {"title": "Target 2.2", "categories": []}, {"title": "Target 2.3", "categories": []}, {"title": "Target 2.4", "categories": []}, {"title": "Target 2.5", "categories": []}, {"title": "Target 2.a", "categories": []}, {"title": "Target 2.b", "categories": []}, {"title": "Target 2.c", "categories": []}]}, {"title": "Goal 3", "categories": [{"title": "Ta

{'success': True}